# Disaster Tweet Classification

Binary classification of disaster-related tweets using TF-IDF and machine learning.

### 1 - Notebook Overview

**Goal:** Predict whether a tweet is about a real disaster (1) or not (0), using the [Kaggle Disaster Tweets](https://www.kaggle.com/c/nlp-getting-started) dataset of 7,613 labeled tweets.

**Pipeline:**

1. Load and inspect the dataset
2. Preprocess tweet text (lowercasing, removing URLs/mentions/special characters, stop word removal, lemmatization)
3. Vectorize text using TF-IDF
4. Train a Logistic Regression baseline model
5. Evaluate with accuracy, precision, recall, F1-score, and confusion matrix
6. Optional: hyperparameter tuning and cross-validation

**Dataset:** `train.csv` with columns `text` (tweet content) and `target` (1 = disaster, 0 = not disaster).